In [ ]:
# ginzaを用いた形態素解析処理とIF-IDFによる特徴量抽出

In [ ]:
!pip install -U ginza ja-ginza
!pip install nlplot

In [ ]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

# データセットをロード
df = pd.read_csv('sample.csv')

# 日本語の言語モデルをロード
nlp = spacy.load("ja_ginza")

In [ ]:
# 追加したいストップワードのリスト
new_stopwords = []

# ストップワードリストに新しい単語を追加
for word in new_stopwords:
    nlp.vocab[word].is_stop = True

In [ ]:
def process_text(text, allowed_postags):
    doc = nlp(text)
    words = [token.lemma_ for token in doc if token.tag_ in allowed_postags and not token.is_stop]
    return " ".join(words)

In [ ]:
# 目次のための形態素
mokuji_postags = ['名詞-普通名詞-一般',]
# 紹介文のための形態素
shoukai_postags = ['名詞-普通名詞-一般', '動詞-一般', '形容詞-一般']

In [ ]:
# '目次' と '紹介文' のカラムを処理
df['processed_目次'] = df['目次'].astype(str).apply(lambda x: process_text(x, mokuji_postags))
df['processed_紹介文'] = df['商品紹介'].astype(str).apply(lambda x: process_text(x, shoukai_postags))

In [ ]:
# テキストから上位のフィーチャを抽出する関数
def extract_top_features(column, top_n=5):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df[column])
    words = vectorizer.get_feature_names_out()

    top_features = []
    for vec in X.toarray():
        top_indices = sorted(range(len(vec)), key=lambda i: vec[i], reverse=True)[:top_n]
        top_features.append([(words[i], vec[i]) for i in top_indices])

    return top_features

In [ ]:
df['目次の上位5特徴語'] = extract_top_features('processed_目次')
df['紹介文の上位5特徴語'] = extract_top_features('processed_紹介文')